# Merging Functional Annotations into the CADD Chr22 File using Python

### Set Working Directory and Import Files / Pandas:

In [28]:
# Check Working Directory.

import os
import sys
print("wd"
     , os.getcwd())

wd /Volumes/HZU/CADD/hg19


In [4]:
# Set Working Directory to Hard Drive, which contains all the needed files.

os.chdir('/Volumes/HZU/CADD/hg19')
print("wd"
     , os.getcwd())

wd /Volumes/HZU/CADD/hg19


In [9]:
# Import Pandas and NumPy

import pandas as pd
import numpy as np

### Import and View Chr22 CADD File

In [38]:
# We run the separator as ‘\t’ because in this file the tab character separates the fields.

ch22 = pd.read_csv("chr22F1-43m.txt",sep='\t', dtype = 'str')

In [39]:
# We can quickly view the data to make sure there are no issues and everything looks normal.

ch22.head()

## CADD GRCh37-v1.6 (c) University of Washington, Hudson-Alpha Institute for Biotechnology and Berlin Institute of Health 2013-2019. All rights reserved.
#Chrom Pos      Ref                                                Alt                                                                                                       
22     16050001 G                                                    A                                                                                                       
                G                                                    C                                                                                                       
                G                                                    T                                                                                                       
       16050002 A                                                    C

In [46]:
print(ch22.columns.tolist())

# This first row will cause a KeyError as left_join will be unable to recognize the correct headers.

['## CADD GRCh37-v1.6 (c) University of Washington, Hudson-Alpha Institute for Biotechnology and Berlin Institute of Health 2013-2019. All rights reserved.']


In [42]:
# Thus, we remove the "Title" Row (through the command line) to Prevent a future KeyError while Left_Joining CADD chr22 with a functional annotation dataset.

ch22nh = pd.read_csv("/Volumes/HZU/CADD/hg19/chr22F1-43mnh.txt",sep='\t', dtype = 'str')
ch22nh.head()

,#Chrom,Pos,Ref,Alt
0,22,16050001,G,A
1,22,16050001,G,C
2,22,16050001,G,T
3,22,16050002,A,C
4,22,16050002,A,G


### Import and View ClinVar Functional Annotations

In [61]:
# Now, we can import the ClinVar data.

clinvar = pd.read_csv('clinvar_20220528.txt', comment='#', sep='\t' , dtype='str')

# Additional bits (comment='#', sep='\t') needed to make the vcf file "readable" to Pandas left_join.

In [62]:
# Let's take a look:

clinvar.head()

,1,861332,1019397,G,A,.,..1,"ALLELEID=1003021;CLNDISDB=MedGen:CN517202;CLNDN=not_provided;CLNHGVS=NC_000001.10:g.861332G>A;CLNREVSTAT=criteria_provided,_single_submitter;CLNSIG=Uncertain_significance;CLNVC=single_nucleotide_variant;CLNVCSO=SO:0001483;GENEINFO=SAMD11:148398;MC=SO:0001583|missense_variant;ORIGIN=1;RS=1640863258"
0,1,861336,1543320,C,T,.,.,ALLELEID=1632777;CLNDISDB=MedGen:CN517202;CLND...
1,1,861349,1648427,C,T,.,.,ALLELEID=1600580;CLNDISDB=MedGen:CN517202;CLND...
2,1,861356,1362713,T,C,.,.,ALLELEID=1396033;CLNDISDB=MedGen:CN517202;CLND...
3,1,861366,1568423,C,T,.,.,ALLELEID=1570515;CLNDISDB=MedGen:CN517202;CLND...
4,1,861383,1365270,C,T,.,.,ALLELEID=1502313;CLNDISDB=MedGen:CN517202;CLND...


In [63]:
# Let's add headers.

clinvar.columns = ["#CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO"]

In [64]:
# Let's take another look:

clinvar.head()

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,1,861336,1543320,C,T,.,.,ALLELEID=1632777;CLNDISDB=MedGen:CN517202;CLND...
1,1,861349,1648427,C,T,.,.,ALLELEID=1600580;CLNDISDB=MedGen:CN517202;CLND...
2,1,861356,1362713,T,C,.,.,ALLELEID=1396033;CLNDISDB=MedGen:CN517202;CLND...
3,1,861366,1568423,C,T,.,.,ALLELEID=1570515;CLNDISDB=MedGen:CN517202;CLND...
4,1,861383,1365270,C,T,.,.,ALLELEID=1502313;CLNDISDB=MedGen:CN517202;CLND...


### Import and View Eigen Functional Annotations

In [21]:
# Now, we repeat the process for the Eigen dataset.

eigen = pd.read_csv("/Volumes/HZU/humandb/headers_hg19_Eigen22.txt",sep='\t', on_bad_lines='skip', dtype='str')
eigen.head()

#Chrom  \
22 16157306 16157306 T A . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  intron_variant,non_coding_transcript_variant   
                       C . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  intron_variant,non_coding_transcript_variant   
                       G . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  intron_variant,non_coding_transcript_variant   
   16157307 16157307 C A . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  intron_variant,non_coding_transcript_variant   
                       G . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  intron_variant,non_coding_transcript_variant   

                                                                                                 Pos  \
22 16157306 16157306 T A . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  -0.723507420492939   
                       C . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  -0.723507420492939   
                       G . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  -0.723507420492939   
   16157307 16157307 C A . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  -0.690256467806581   
                       G . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  -0.690256467806581   

                                                                                       Pos2  \
22 16157306 16157306 T A . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.7735336   
                       C . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.7735336   
                       G . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.7735336   
   16157307 16157307 C A . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049   0.829126   
                       G . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049   0.829126   

                                                                                                 Ref  \
22 16157306 16157306 T A . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043   -0.95435275448397   
                       C . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043   -0.95435275448397   
                       G . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043   -0.95435275448397   
   16157307 16157307 C A . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  -0.919775085203436   
                       G . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  -0.919775085203436   

                                                                                        Alt  
22 16157306 16157306 T A . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.5475722  
                       C . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.5475722  
                       G . . . . 0.511 0.511 0.234 0.476 0.153 0.015 0.019 0.043  0.5475722  
   16157307 16157307 C A . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  0.5935479  
                       G . . . . 0.511 0.511 0.282 0.569 0.680 0.016 0.022 0.049  0.5935479

### Import and View dbnsfp33a Functional Annotations

In [24]:
# Finally, the dbnsfp33a annotations:
    
dbnsfp33a = pd.read_csv("/Volumes/HZU/humandb/headers_hg19_dbnsfp33a22.txt",sep='\t', on_bad_lines='skip' , dtype='str')
dbnsfp33a.head()

#Chrom  \
22 16287549 16287549 G A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.001 0.002 T . . . . . . . . c -0.501 -0.817 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016      .   
                       C . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016      .   
                       T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.000 0.000 T . . . . . . . . c -1.079 -1.226 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016      .   
   16287550 16287550 C A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . n -0.638 -0.264 0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016      .   
                       G . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016      .   

                                                                                                                                                                                                                       Pos  \
22 16287549 16287549 G A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.001 0.002 T . . . . . . . . c -0.501 -0.817 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016   .   
                       C . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016   .   
                       T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.000 0.000 T . . . . . . . . c -1.079 -1.226 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016   .   
   16287550 16287550 C A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . n -0.638 -0.264 0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016   .   
                       G . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016   .   

                                                                                                                                                                                                                       Pos2  \
22 16287549 16287549 G A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.001 0.002 T . . . . . . . . c -0.501 -0.817 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016    .   
                       C . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016    .   
                       T . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 0.000 0.000 T . . . . . . . . c -1.079 -1.226 0.000 0.012 0.487 0.133 0 . . 1.051  0.298 0.179  0.247 0.001 0.137 0.000 0.016    .   
   16287550 16287550 C A . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . n -0.638 -0.264 0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016    .   
                       G . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .     .     . . . . . . . . . . .      .      0.000 0.012 0.487 0.133 0 . . -4.314 0.003 -3.199 0.001 0.000 0.063 0.000 0.016    .   

                                                                                                                                                                                                

###  Merge the CADD Ch22 File with Eigen First to create a "Ch22Eigen" File

In [43]:
ch22eigen = pd.merge(ch22nh, eigen, on =['#Chrom', 'Pos', 'Ref', 'Alt'], how="left")
ch22eigen

,#Chrom,Pos,Ref,Alt,Pos2
0,22,16050001,G,A,NaN
1,22,16050001,G,C,NaN
2,22,16050001,G,T,NaN
3,22,16050002,A,C,NaN
4,22,16050002,A,G,NaN
...,...,...,...,...,...
2999993,22,16929617,A,T,NaN
2999994,22,16929618,T,A,NaN
2999995,22,16929618,T,C,NaN
2999996,22,16929618,T,G,NaN


### Now incorporate the dbnsfp33a data into this "Ch22Eigen" data:

In [50]:
ch22_eigen_dbn = pd.merge(ch22eigen, dbnsfp33a, on =['#Chrom', 'Pos', 'Ref', 'Alt'], how="left")
ch22_eigen_dbn

,#Chrom,Pos,Ref,Alt,Pos2_x,Pos2_y
0,22,16050001,G,A,NaN,NaN
1,22,16050001,G,C,NaN,NaN
2,22,16050001,G,T,NaN,NaN
3,22,16050002,A,C,NaN,NaN
4,22,16050002,A,G,NaN,NaN
...,...,...,...,...,...,...
2999993,22,16929617,A,T,NaN,NaN
2999994,22,16929618,T,A,NaN,NaN
2999995,22,16929618,T,C,NaN,NaN
2999996,22,16929618,T,G,NaN,NaN


### Merge in the ClinVar data to the file already containing CADD, Eigen, and dbnsfp33a data.

In [65]:
ch22_eigen_dbn_clinvar = pd.merge(ch22_eigen_dbn, clinvar, left_on=['#Chrom', 'Pos', 'Ref', 'Alt'], right_on=['#CHROM', 'POS', 'REF', 'ALT'], how="left")
ch22_eigen_dbn_clinvar

,#Chrom,Pos,Ref,Alt,Pos2_x,Pos2_y,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,22,16050001,G,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,16050001,G,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22,16050001,G,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,16050002,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,16050002,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999993,22,16929617,A,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999994,22,16929618,T,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999995,22,16929618,T,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2999996,22,16929618,T,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
